In [1]:
%pip install azure-ai-ml azure-identity

Note: you may need to restart the kernel to use updated packages.


In [2]:
# deployment
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

In [3]:
# enter details of your Azure Machine Learning workspace
subscription_id = ""
resource_group = ""
workspace = ""

In [4]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [5]:
import datetime

endpoint_name = "lugtts-endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="This is the endpoint for inferencing text to speech in luganda",
    auth_mode="key"
)

In [6]:
model = Model(path="Conda.yaml")
env = Environment(
    conda_file="Conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
)

# This deployment was tested with the following instance type "STANDARD_DS3_V2", make sure you have enough quota in your subscription
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="Inference", scoring_script="Score.py"
    ),
    instance_type="STANDARD_DS3_V2",
    instance_count=1,
)

In [7]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Creating local endpoint (lugtts-endpt-10211543386167) .Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'lugtts-endpt-10211543386167', 'description': 'This is the endpoint for inferencing text to speech in luganda', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/home/azureuser/.azureml/inferencing/lugtts-endpt-10211543386167'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f54f35b34c0>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [8]:
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Creating local deployment (lugtts-endpt-10211543386167 / blue) .
Building Docker image from Dockerfile
Step 1/6 : FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
 ---> a79caa0393e2
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> 88d58a5cddf4
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Using cache
 ---> da749315a1c6
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> Using cache
 ---> 8f6f77516a02
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Using cache
 ---> d00eb229ddd9
Step 6/6 : CMD ["conda", "run", "--no-capture-output", "-n", "inf-conda-env", "runsvdir", "/var/runit"]
 ---> Using cache
 ---> 5cf8af62c64b
Successfully built 5cf8af62c64b
Successfully tagged lugtts-endpt-10211543386167:blue

Starting up endpoint...Done (0m 20s)


ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'lugtts-endpt-10211543386167', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/translationinference/code/Users/2100800280/OasisAiTranslation'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f54f0c0e050>, 'model': Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': '2b32fce99043842ba7d7d50372495ea0', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/translationinference/code/Users/2100800280/OasisAiTranslation'), 'creation_context': 

In [9]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, local=True, lines=500
)

'2023-10-21T15:44:00,739887836+00:00 - rsyslog/run \r\n2023-10-21T15:44:00,744611963+00:00 - gunicorn/run \r\n2023-10-21T15:44:00,746073395+00:00 | gunicorn/run | \r\n2023-10-21T15:44:00,747756847+00:00 | gunicorn/run | ###############################################\r\n2023-10-21T15:44:00,749406896+00:00 | gunicorn/run | AzureML Container Runtime Information\r\n2023-10-21T15:44:00,750984839+00:00 | gunicorn/run | ###############################################\r\n2023-10-21T15:44:00,752652689+00:00 | gunicorn/run | \r\n2023-10-21T15:44:00,772795010+00:00 - nginx/run \r\n2023-10-21T15:44:01,269159769+00:00 | gunicorn/run | \r\n2023-10-21T15:44:01,276658046+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20230227.v3\r\n2023-10-21T15:44:01,278277093+00:00 | gunicorn/run | \r\n2023-10-21T15:44:01,279749226+00:00 | gunicorn/run | \r\n2023-10-21T15:44:01,281351370+00:00 | gunicorn/run | PATH environment variable: /opt/miniconda/envs/inf-conda

In [10]:
ml_client.online_endpoints.get(name=endpoint_name, local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32770/score', 'openapi_uri': None, 'name': 'lugtts-endpt-10211543386167', 'description': 'This is the endpoint for inferencing text to speech in luganda', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/translationinference/code/Users/2100800280/OasisAiTranslation'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f54f35b3130>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [11]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, local=True, lines=500
)

'2023-10-21T15:44:00,739887836+00:00 - rsyslog/run \r\n2023-10-21T15:44:00,744611963+00:00 - gunicorn/run \r\n2023-10-21T15:44:00,746073395+00:00 | gunicorn/run | \r\n2023-10-21T15:44:00,747756847+00:00 | gunicorn/run | ###############################################\r\n2023-10-21T15:44:00,749406896+00:00 | gunicorn/run | AzureML Container Runtime Information\r\n2023-10-21T15:44:00,750984839+00:00 | gunicorn/run | ###############################################\r\n2023-10-21T15:44:00,752652689+00:00 | gunicorn/run | \r\n2023-10-21T15:44:00,772795010+00:00 - nginx/run \r\n2023-10-21T15:44:01,269159769+00:00 | gunicorn/run | \r\n2023-10-21T15:44:01,276658046+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20230227.v3\r\n2023-10-21T15:44:01,278277093+00:00 | gunicorn/run | \r\n2023-10-21T15:44:01,279749226+00:00 | gunicorn/run | \r\n2023-10-21T15:44:01,281351370+00:00 | gunicorn/run | PATH environment variable: /opt/miniconda/envs/inf-conda

In [12]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="sample-request.json",
    local=True,
)

'"Museven ruled this country."'

In [13]:
endpoint = ml_client.online_endpoints.get(endpoint_name, local=True)
endpoint

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32770/score', 'openapi_uri': None, 'name': 'lugtts-endpt-10211543386167', 'description': 'This is the endpoint for inferencing text to speech in luganda', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/translationinference/code/Users/2100800280/OasisAiTranslation'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f54f0c5bac0>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [14]:
# Deploying online endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint)

In [17]:
ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint lugtts-endpt-10211543386167 exists


...

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint)

.

HttpResponseError: (BadRequest) The request is invalid.
Code: BadRequest
Message: The request is invalid.
Exception Details:	(InferencingClientCallFailed) {{"errors":{{"DeploymentWeights":["Deployments given positive traffic values should be either in a successful or failed state. Unmatched deployments: [blue]"]}},"title":"One or more validation errors occurred."}}
	Code: InferencingClientCallFailed
	Message: {{"errors":{{"DeploymentWeights":["Deployments given positive traffic values should be either in a successful or failed state. Unmatched deployments: [blue]"]}},"title":"One or more validation errors occurred."}}
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "262ac792d6a1a2e507c6ff1c559bede5",
        "request": "2ca0dff51c697669"
    }
}Type: Environment
Info: {
    "value": "eastus2"
}Type: Location
Info: {
    "value": "eastus2"
}Type: Time
Info: {
    "value": "2023-10-21T15:5

In [ ]:
ml_client.online_endpoints.get(name=endpoint_name)

In [ ]:
for endpoint in ml_client.online_endpoints.list():
    print(endpoint.name)

In [ ]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, lines=50
)

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="sample-request.json",
)

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)

In [ ]:
token =  ml_client.online_endpoints.get_keys(name=endpoint_name).primary_key
